<a href="https://colab.research.google.com/github/nathencinoza/machine-learning/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import top_k_accuracy_score 

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Orga 2/TP3/TP3 dataset music/train.parquet')
df.head(5)

,track_name,lyric,genre,language,popularity,artist,a_genres,a_songs,a_popularity,did,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Halo,Remember those walls I built\nWell baby they'r...,Dance,en,79,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
1,Halo,Remember those walls I built\nWell baby they'r...,Pop,en,80,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
2,Halo,Remember those walls I built\nWell baby they'r...,R&B,en,80,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
3,Partition,Driver roll up the partition please\nDriver ro...,Dance,en,71,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174
4,Partition,Driver roll up the partition please\nDriver ro...,Pop,en,71,Beyoncé,Pop; R&B; Black Music,276.0,205.5,NaN,...,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174


Elimino columnas que no voy a usar

In [ ]:
df.drop(['track_name', 'did', 'a_genres'], axis = 1, inplace=True)

Separo el set de train con 80% de los artistas y el set de validación con el 20% de artistas restantes de manera aleatoria. Son 1072 artistas, 857 para train y 215 para valid. Y elimino la columna de artistas para ambos sets de datos.

In [ ]:
from sklearn.model_selection import GroupShuffleSplit 
splitter = GroupShuffleSplit(test_size=0.2, n_splits=2, random_state = 142)
split = splitter.split(df, groups=df['artist'])
train_index, valid_index = next(split)

train = df.iloc[train_index]
valid = df.iloc[valid_index]

In [ ]:
df['artist'].nunique(), train['artist'].nunique(), valid['artist'].nunique() 

(1072, 857, 215)

In [ ]:
train.drop('artist', axis=1, inplace=True)
valid.drop('artist', axis=1, inplace=True)

In [ ]:
train.head(5)

,lyric,genre,language,popularity,a_songs,a_popularity,s-label,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Remember those walls I built\nWell baby they'r...,Dance,en,79,276.0,205.5,NaN,0.2720,0.508,261640,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
1,Remember those walls I built\nWell baby they'r...,Pop,en,80,276.0,205.5,NaN,0.2720,0.508,261640,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
2,Remember those walls I built\nWell baby they'r...,R&B,en,80,276.0,205.5,NaN,0.2720,0.508,261640,0.720,0.0000,B,0.0563,-5.908,Minor,0.0628,79.983,4/4,0.472
3,Driver roll up the partition please\nDriver ro...,Dance,en,71,276.0,205.5,NaN,0.0296,0.412,319467,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174
4,Driver roll up the partition please\nDriver ro...,Pop,en,71,276.0,205.5,NaN,0.0296,0.412,319467,0.441,0.0726,B,0.3060,-11.523,Minor,0.2910,185.571,4/4,0.174


In [ ]:
valid.head(5)

,lyric,genre,language,popularity,a_songs,a_popularity,s-label,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
143,"And you got me, like oh\nWhat you want from me...",Dance,en,73,246.0,51.2,NaN,0.07170,0.509,224000,0.637,0.000011,E,0.0789,-4.83,Minor,0.0471,172.006,3/4,0.378
144,"And you got me, like oh\nWhat you want from me...",Pop,en,74,246.0,51.2,NaN,0.07170,0.509,224000,0.637,0.000011,E,0.0789,-4.83,Minor,0.0471,172.006,3/4,0.378
145,"And you got me, like oh\nWhat you want from me...",R&B,en,74,246.0,51.2,NaN,0.07170,0.509,224000,0.637,0.000011,E,0.0789,-4.83,Minor,0.0471,172.006,3/4,0.378
146,Shine bright like a diamond\nShine bright like...,Dance,en,64,246.0,51.2,NaN,0.00125,0.564,225147,0.710,0.000000,B,0.1090,-4.92,Minor,0.0461,91.972,4/4,0.393
147,Shine bright like a diamond\nShine bright like...,Pop,en,65,246.0,51.2,NaN,0.00125,0.564,225147,0.710,0.000000,B,0.1090,-4.92,Minor,0.0461,91.972,4/4,0.393


Para deshacernos de los NaN:

* Para la columna de s-label los cambiamos por el promedio de la columna
* Para la columna language los cambiamos por el leguaje  más común
* Para la columna lyrics los llenamos con espacios vacíos. 

In [ ]:
train['lyric'] = train['lyric'].fillna("")
valid['lyric'] = valid['lyric'].fillna("")

train['language'] = train['language'].fillna(train['language'].value_counts().idxmax())
valid['language'] = valid['language'].fillna(train['language'].value_counts().idxmax())

train['s-label'] = train['s-label'].fillna(train['s-label'].mean())
valid['s-label'] = valid['s-label'].fillna(train['s-label'].mean())

# Encoding

Como la columna de key tiene un orden (c = 0, c# = 1, etc), cambiamos los valores de string por int. 

In [ ]:
train['key'] = train['key'].apply(lambda x: 0 if (x == 'C') else (1 if (x == 'C#') else (2 if (x == 'D') else (3 if (x == 'D#') else (4 if (x == 'E') else (5 if (x == 'F') else (6 if (x == 'F#') else (7 if (x == 'G') else (8 if (x == 'G#') else (9 if (x == 'A') else (10 if (x == 'A#') else 11)))))))))))
valid['key'] = valid['key'].apply(lambda x: 0 if (x == 'C') else (1 if (x == 'C#') else (2 if (x == 'D') else (3 if (x == 'D#') else (4 if (x == 'E') else (5 if (x == 'F') else (6 if (x == 'F#') else (7 if (x == 'G') else (8 if (x == 'G#') else (9 if (x == 'A') else (10 if (x == 'A#') else 11)))))))))))

Para mode y time_signature utilizaremos One Hot Encoding, ya que los diferentes valores de cada columna son pocos

In [ ]:
df['mode'].nunique(), df['time_signature'].nunique()

(2, 4)

In [ ]:
time_signature_oneHotEncoder = OneHotEncoder(drop='first', handle_unknown='ignore')

encoded_time_signature = time_signature_oneHotEncoder.fit_transform(train[['time_signature']]).todense().astype(int)
train = train.reset_index().drop(['index', 'time_signature'],axis=1).join(pd.DataFrame(encoded_time_signature, columns= time_signature_oneHotEncoder.get_feature_names_out()))

encoded_time_signature_valid = time_signature_oneHotEncoder.transform(valid[['time_signature']]).todense().astype(int)
valid = valid.reset_index().drop(['index', 'time_signature'], axis=1).join(pd.DataFrame(encoded_time_signature_valid, columns= time_signature_oneHotEncoder.get_feature_names_out()))

In [ ]:
mode_oneHotEncoder = OneHotEncoder(drop='first', handle_unknown='ignore')

encoded_mode = mode_oneHotEncoder.fit_transform(train[['mode']]).todense().astype(int)
train = train.reset_index().drop(['index', 'mode'],axis=1).join(pd.DataFrame(encoded_mode, columns= mode_oneHotEncoder.get_feature_names_out()))

encoded_mode_valid = mode_oneHotEncoder.transform(valid[['mode']]).todense().astype(int)
valid = valid.reset_index().drop(['index', 'mode'], axis=1).join(pd.DataFrame(encoded_mode_valid, columns= mode_oneHotEncoder.get_feature_names_out()))

Aplicamos Mean Encoding a language

In [ ]:
train['language'] =  train.groupby('language')['energy'].transform('mean')
valid['language'] =  train.groupby('language')['energy'].transform('mean')

Y por último aplicamos Count Vectorizer para lyrics

In [ ]:
vectorizer = CountVectorizer(lowercase=True, stop_words='english', max_features=40)
transformation = vectorizer.fit_transform(train['lyric'])

vectorizer_df = pd.DataFrame(transformation.todense(), columns = vectorizer.get_feature_names_out())
train = pd.concat([train, vectorizer_df], axis=1)
train.drop('lyric', axis=1, inplace=True)
train.head()

,genre,language,popularity,a_songs,a_popularity,s-label,acousticness,danceability,duration_ms,energy,...,say,tell,think,time,ve,wanna,want,way,won,yeah
0,Dance,0.674579,79,276.0,205.5,0.485105,0.2720,0.508,261640,0.720,...,0,0,1,0,2,0,1,1,2,0
1,Pop,0.674579,80,276.0,205.5,0.485105,0.2720,0.508,261640,0.720,...,0,0,1,0,2,0,1,1,2,0
2,R&B,0.674579,80,276.0,205.5,0.485105,0.2720,0.508,261640,0.720,...,0,0,1,0,2,0,1,1,2,0
3,Dance,0.674579,71,276.0,205.5,0.485105,0.0296,0.412,319467,0.441,...,0,0,0,0,0,6,1,0,0,1
4,Pop,0.674579,71,276.0,205.5,0.485105,0.0296,0.412,319467,0.441,...,0,0,0,0,0,6,1,0,0,1


In [ ]:
transformation = vectorizer.transform(valid['lyric'])
vectorizer_valid = pd.DataFrame(transformation.todense(), columns= vectorizer.get_feature_names_out())
valid = pd.concat([valid, vectorizer_valid], axis=1)
valid.drop('lyric', axis=1, inplace=True)
valid.head()

,genre,language,popularity,a_songs,a_popularity,s-label,acousticness,danceability,duration_ms,energy,...,say,tell,think,time,ve,wanna,want,way,won,yeah
0,Dance,0.674579,73,246.0,51.2,0.485105,0.07170,0.509,224000,0.637,...,0,0,0,0,0,0,2,2,0,4
1,Pop,0.674579,74,246.0,51.2,0.485105,0.07170,0.509,224000,0.637,...,0,0,0,0,0,0,2,2,0,4
2,R&B,0.674579,74,246.0,51.2,0.485105,0.07170,0.509,224000,0.637,...,0,0,0,0,0,0,2,2,0,4
3,Dance,0.674579,64,246.0,51.2,0.485105,0.00125,0.564,225147,0.710,...,0,0,0,0,0,0,0,0,0,0
4,Pop,0.674579,65,246.0,51.2,0.485105,0.00125,0.564,225147,0.710,...,0,0,0,0,0,0,0,0,0,0


# XGBoost

Separamos y dejamos en x_ las features a usar, y en y_ el género que es lo que queremos predecir, entrenamos nuestro modelo y buscamos los mejores hiperparametros.

In [ ]:
x_train = train.drop('genre',axis=1)
y_train = train['genre']

x_valid = valid.drop('genre',axis=1)
y_valid = valid['genre']

In [ ]:
xgb_classifier = xgb.XGBClassifier(objective = 'multi:softmax', num_class = 26, random_state = 115)
xgb_classifier.fit(x_train, y_train)

XGBClassifier(num_class=26, objective='multi:softprob', random_state=115)

In [ ]:
hiperparametros = {
    "max_depth": [4, 5, 6, 7, 8],
    "learning_rate" : [0.1, 0.2, 0.3, 0.4],
    "colsample_bytree" : [0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [1, 2, 3, 4, 5]
}
model = RandomizedSearchCV(xgb_classifier, hiperparametros,  n_iter = 5, scoring="accuracy", random_state=115)
model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


RandomizedSearchCV(estimator=XGBClassifier(num_class=26,
                                           objective='multi:softprob',
                                           random_state=115),
                   n_iter=5,
                   param_distributions={'colsample_bytree': [0.1, 0.2, 0.3,
                                                             0.4],
                                        'learning_rate': [0.1, 0.2, 0.3, 0.4],
                                        'max_depth': [4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 3, 4, 5]},
                   random_state=115, scoring='accuracy')

Mejor score de validación:

In [ ]:
top_k_accuracy_score(y_valid, model.predict_proba(x_valid), k = 2,normalize = True, labels = model.classes_)

0.49361259655377304

Mejores hiperparametros: los parametros default de XGBoost


In [ ]:
top_k_accuracy_score(y_valid, xgb_classifier.predict_proba(x_valid), k = 2,normalize = True, labels = xgb_classifier.classes_)

0.5355020796197266